In [1]:
############ ML_model_SVM ##################

In [2]:
# Update sklearn to prevent version mismatches
# !pip install sklearn --upgrade

#### Data preparation 

In [3]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv("../Data/ksi_cleaned.csv")
df.head()

,ACCNUM,DATE,TIME,HOUR,STREET1,STREET2,ROAD_CLASS,LATITUDE,LONGITUDE,LOCCOORD,...,TRUCK,TRSN_CITY_VEH,EMERG_VEH,SPEEDING,AG_DRIV,REDLIGHT,ALCOHOL,DISABILITY,Hood_ID,Neighbourhood
0,893184,2006/01/01 05:00:00+00,236,2,WOODBINE AVE,O CONNOR DR,Major Arterial,43.699595,-79.318797,Intersection,...,No,No,No,Yes,Yes,No,Yes,No,60,Woodbine-Lumsden (60)
1,893184,2006/01/01 05:00:00+00,236,2,WOODBINE AVE,O CONNOR DR,Major Arterial,43.699595,-79.318797,Intersection,...,No,No,No,Yes,Yes,No,Yes,No,60,Woodbine-Lumsden (60)
2,893184,2006/01/01 05:00:00+00,236,2,WOODBINE AVE,O CONNOR DR,Major Arterial,43.699595,-79.318797,Intersection,...,No,No,No,Yes,Yes,No,Yes,No,60,Woodbine-Lumsden (60)
3,893184,2006/01/01 05:00:00+00,236,2,WOODBINE AVE,O CONNOR DR,Major Arterial,43.699595,-79.318797,Intersection,...,No,No,No,Yes,Yes,No,Yes,No,60,Woodbine-Lumsden (60)
4,893184,2006/01/01 05:00:00+00,236,2,WOODBINE AVE,O CONNOR DR,Major Arterial,43.699595,-79.318797,Intersection,...,No,No,No,Yes,Yes,No,Yes,No,60,Woodbine-Lumsden (60)


In [5]:
df.columns

Index(['ACCNUM', 'DATE', 'TIME', 'HOUR', 'STREET1', 'STREET2', 'ROAD_CLASS',
       'LATITUDE', 'LONGITUDE', 'LOCCOORD', 'TRAFFCTL', 'VISIBILITY', 'LIGHT',
       'RDSFCOND', 'ACCLASS', 'INVTYPE', 'INVAGE', 'Age_mid', 'INJURY',
       'VEHTYPE', 'DRIVACT', 'DRIVCOND', 'PEDESTRIAN', 'CYCLIST', 'AUTOMOBILE',
       'MOTORCYCLE', 'TRUCK', 'TRSN_CITY_VEH', 'EMERG_VEH', 'SPEEDING',
       'AG_DRIV', 'REDLIGHT', 'ALCOHOL', 'DISABILITY', 'Hood_ID',
       'Neighbourhood'],
      dtype='object')

In [6]:
# Select features then Assign X (data) and y (target)
X = df[["HOUR", "ROAD_CLASS" , "TRAFFCTL", "VISIBILITY", "LIGHT", "RDSFCOND"]]
y = df["ACCLASS"]
print(X.shape, y.shape)

(16093, 6) (16093,)


In [7]:
y.head()

0    Non-Fatal Injury
1    Non-Fatal Injury
2    Non-Fatal Injury
3    Non-Fatal Injury
4    Non-Fatal Injury
Name: ACCLASS, dtype: object

In [8]:
#features : all categorical value
X.columns 

Index(['HOUR', 'ROAD_CLASS', 'TRAFFCTL', 'VISIBILITY', 'LIGHT', 'RDSFCOND'], dtype='object')

#### Preprocessing

Transform to numeric representation and add as a new columns

In [9]:
from sklearn.preprocessing import LabelEncoder,OneHotEncoder

#0. HOUR
HOUR_LE = LabelEncoder()
HOUR_labels = HOUR_LE.fit_transform(X.HOUR)
X["HOUR_Label"]= HOUR_labels

#1.'ROAD_CLASS'
Road_class_LE = LabelEncoder()
Road_class_labels = Road_class_LE.fit_transform(X.ROAD_CLASS)
X["ROADCLASS_Label"]=Road_class_labels

#2.'TRAFFCTL'
TRAFFCTL_LE = LabelEncoder()
TRAFFCTL_labels = TRAFFCTL_LE.fit_transform(X.TRAFFCTL)
X["TRAFFCTL_Label"]=TRAFFCTL_labels

#3.'VISIBILITY'
VISIBILITY_LE = LabelEncoder()
VISIBILITY_labels = VISIBILITY_LE.fit_transform(X.VISIBILITY)
X["VISIBILITY_Label"]=VISIBILITY_labels

#4.'LIGHT'
LIGHT_LE = LabelEncoder()
LIGHT_labels = LIGHT_LE.fit_transform(X.LIGHT)
X["LIGHT_Label"]=LIGHT_labels

#5.'RDSFCOND'
RDSFCOND_LE = LabelEncoder()
RDSFCOND_labels = RDSFCOND_LE.fit_transform(X.RDSFCOND)
X["RDSFCOND_Label"]=RDSFCOND_labels

X.head()

C:\Users\JJ\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\JJ\anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\JJ\anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats i

,HOUR,ROAD_CLASS,TRAFFCTL,VISIBILITY,LIGHT,RDSFCOND,HOUR_Label,ROADCLASS_Label,TRAFFCTL_Label,VISIBILITY_Label,LIGHT_Label,RDSFCOND_Label
0,2,Major Arterial,No Control,Clear,Dark,Wet,2,4,0,0,0,9
1,2,Major Arterial,No Control,Clear,Dark,Wet,2,4,0,0,0,9
2,2,Major Arterial,No Control,Clear,Dark,Wet,2,4,0,0,0,9
3,2,Major Arterial,No Control,Clear,Dark,Wet,2,4,0,0,0,9
4,2,Major Arterial,No Control,Clear,Dark,Wet,2,4,0,0,0,9


In [10]:
#X.to_csv("X_label_encoded_byAcc.csv")

####### one-hot encoding scheme easily by leveraging the to_dummies(…) function from pandas.

In [11]:
# 'HOUR' ,'ROAD_CLASS', 'TRAFFCTL', 'VISIBILITY', 'LIGHT', 'RDSFCOND'

HOUR_onehot_features = pd.get_dummies(X['HOUR'])
ROADCLASS_onehot_features = pd.get_dummies(X['ROAD_CLASS'])
TRAFFCTL_onehot_features = pd.get_dummies(X['TRAFFCTL'])
VISIBILITY_onehot_features = pd.get_dummies(X['VISIBILITY'])
LIGHT_onehot_features = pd.get_dummies(X['LIGHT'])
RDSFCOND_onehot_features = pd.get_dummies(X['RDSFCOND'])

# check to see if it works
RDSFCOND_onehot_features.head()

,Dry,Ice,Loose Sand or Gravel,Loose Snow,Other,Packed Snow,Slush,Spilled liquid,Unknown,Wet
0,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,1


In [12]:
# combine all dataframe (features) into one df
X_all_features = pd.concat([X, HOUR_onehot_features, ROADCLASS_onehot_features,TRAFFCTL_onehot_features,VISIBILITY_onehot_features,
                           LIGHT_onehot_features,RDSFCOND_onehot_features],axis=1)
X_all_features.head()

,HOUR,ROAD_CLASS,TRAFFCTL,VISIBILITY,LIGHT,RDSFCOND,HOUR_Label,ROADCLASS_Label,TRAFFCTL_Label,VISIBILITY_Label,...,Dry,Ice,Loose Sand or Gravel,Loose Snow,Other,Packed Snow,Slush,Spilled liquid,Unknown,Wet
0,2,Major Arterial,No Control,Clear,Dark,Wet,2,4,0,0,...,0,0,0,0,0,0,0,0,0,1
1,2,Major Arterial,No Control,Clear,Dark,Wet,2,4,0,0,...,0,0,0,0,0,0,0,0,0,1
2,2,Major Arterial,No Control,Clear,Dark,Wet,2,4,0,0,...,0,0,0,0,0,0,0,0,0,1
3,2,Major Arterial,No Control,Clear,Dark,Wet,2,4,0,0,...,0,0,0,0,0,0,0,0,0,1
4,2,Major Arterial,No Control,Clear,Dark,Wet,2,4,0,0,...,0,0,0,0,0,0,0,0,0,1


In [13]:
#X_all_features.to_csv("X_all_features.csv")

In [14]:
#second data cleaning job
# drop categorical columns(features) and leave its numeric features. Others and Unknown also dropped

X_all_features_cleaned = X_all_features.drop(["Other","Unknown","HOUR","HOUR_Label","ROAD_CLASS","TRAFFCTL","VISIBILITY",
                                              "LIGHT","RDSFCOND",'ROADCLASS_Label', 'TRAFFCTL_Label',
                                              'VISIBILITY_Label','LIGHT_Label','RDSFCOND_Label'],axis=1)

In [15]:
X_all_features_cleaned.columns 

Index([                       0,                        1,
                              2,                        3,
                              4,                        5,
                              6,                        7,
                              8,                        9,
                             10,                       11,
                             12,                       13,
                             14,                       15,
                             16,                       17,
                             18,                       19,
                             20,                       21,
                             22,                       23,
                    'Collector',             'Expressway',
                      'Laneway',                  'Local',
               'Major Arterial',    'Major Arterial Ramp',
               'Minor Arterial',                'Pending',
                   'No Control',           'PXO - No Ped

In [16]:
print(f"Number of features are : {len(X_all_features_cleaned.columns)}") 

Number of features are : 66


In [17]:
#X_all_features_cleaned.to_csv("X_all_features_cleaned.csv" ) 

In [18]:
X_all_features_cleaned

,0,1,2,3,4,5,6,7,8,9,...,Dusk,"Dusk, artificial",Dry,Ice,Loose Sand or Gravel,Loose Snow,Packed Snow,Slush,Spilled liquid,Wet
0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16088,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
16089,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
16090,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
16091,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [19]:
X_features = X_all_features_cleaned

In [20]:
print(X_features.shape)
print(y.shape)

(16093, 66)
(16093,)


In [21]:
y

0        Non-Fatal Injury
1        Non-Fatal Injury
2        Non-Fatal Injury
3        Non-Fatal Injury
4        Non-Fatal Injury
               ...       
16088    Non-Fatal Injury
16089    Non-Fatal Injury
16090    Non-Fatal Injury
16091    Non-Fatal Injury
16092    Non-Fatal Injury
Name: ACCLASS, Length: 16093, dtype: object

#### MODELING : SVM

In [22]:
#Split our data into training and testing
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_features, y, random_state=1)

In [23]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical
# Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

C:\Users\JJ\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\JJ\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\JJ\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\JJ\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1)

In [24]:
# Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [25]:
#Create SVM Model
from sklearn.svm import SVC 
Fatality_classifier = SVC(kernel='linear')

In [27]:
#print(y.shape)
#print(y_train.shape)
#print(encoded_y_train.shape)
#print(y_train_categorical.shape)

In [28]:
#Fit train data to model
Fatality_classifier.fit(X_train, encoded_y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [29]:
#Validate the model using the test data
print(f"Training score (R-square) : {round(Fatality_classifier.score(X_train, encoded_y_train)*100,4)}%")
print(f"Test score (R-square): {round(Fatality_classifier.score(X_test, encoded_y_test)*100,4)}%")

Training score (R-square) : 86.4612%
Test score (R-square): 86.4811%


Make predictions

In [30]:
predictions = Fatality_classifier.predict(X_test)
prediction_labels = label_encoder.inverse_transform(predictions)
result_df=pd.DataFrame({"Prediction": prediction_labels, "Actual": y_test})
result_df

,Prediction,Actual
5633,Non-Fatal Injury,Non-Fatal Injury
3745,Non-Fatal Injury,Fatal
3497,Non-Fatal Injury,Non-Fatal Injury
2305,Non-Fatal Injury,Non-Fatal Injury
14492,Non-Fatal Injury,Fatal
...,...,...
13839,Non-Fatal Injury,Non-Fatal Injury
913,Non-Fatal Injury,Fatal
2588,Non-Fatal Injury,Non-Fatal Injury
3308,Non-Fatal Injury,Non-Fatal Injury


In [31]:
result_df.loc[result_df["Prediction"]=="Fatal"].count()

Prediction    25
Actual        25
dtype: int64

In [32]:
result_df.to_csv("Prediction_results/prediction_result_SVM.csv" ) 

Confusion matrix

In [33]:
# Create Confusion matrix

from sklearn.metrics import confusion_matrix

pd.DataFrame(
    confusion_matrix(y_test, prediction_labels),
    columns=['Prediction:Fatal', 'Prediction:Non-Fatal'],
    index=['Actual:Fatal', 'Actual:Non-Fatal']
)

,Prediction:Fatal,Prediction:Non-Fatal
Actual:Fatal,12,531
Actual:Non-Fatal,13,3468


In [34]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, prediction_labels)

0.8648111332007953

Classification report

In [35]:
# Calculate classification report
from sklearn.metrics import classification_report
print("SVM model ------------------------------")
print(classification_report(y_test, prediction_labels,
                            target_names=["Fatal", "Non-Fatal Injury"]))

SVM model ------------------------------
                  precision    recall  f1-score   support

           Fatal       0.48      0.02      0.04       543
Non-Fatal Injury       0.87      1.00      0.93      3481

        accuracy                           0.86      4024
       macro avg       0.67      0.51      0.48      4024
    weighted avg       0.81      0.86      0.81      4024



In [36]:
#save model
import pickle
filename = 'SVM_final_model.pickle'
pickle.dump(Fatality_classifier, open(filename, 'wb'))

Model tuning

In [99]:
# get the model parameters
print(Fatality_classifier.get_params())

{'C': 1.0, 'break_ties': False, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 'scale', 'kernel': 'linear', 'max_iter': -1, 'probability': False, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}


In [100]:
# Create the GridSearchCV model

from sklearn.model_selection import GridSearchCV

# create parameter grid
#param_grid2 = {'C': [0.1, 1, 5, 10, 50, 100],
#              'gamma': [0.00001, 0.0001, 0.001, 0.01, 0.1, 0.9]}

param_grid = {'C': [0.1, 1, 5, 10],
             'gamma': [0.00001, 0.0001, 0.001, 0.01]}

grid = GridSearchCV(Fatality_classifier, param_grid, verbose=4, scoring='recall')

In [102]:
# Train the model with GridSearch
grid.fit(X_train, encoded_y_train)

Fitting 5 folds for each of 16 candidates, totalling 80 fits
[CV] C=0.1, gamma=1e-05 ..............................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] .................. C=0.1, gamma=1e-05, score=1.000, total=   7.8s
[CV] C=0.1, gamma=1e-05 ..............................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    7.7s remaining:    0.0s


[CV] .................. C=0.1, gamma=1e-05, score=1.000, total=   8.2s
[CV] C=0.1, gamma=1e-05 ..............................................


...

[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:   23.7s remaining:    0.0s


[CV] .................. C=0.1, gamma=1e-05, score=1.000, total=   7.6s
[CV] C=0.1, gamma=1e-05 ..............................................
[CV] .................. C=0.1, gamma=1e-05, score=1.000, total=   8.2s
[CV] C=0.1, gamma=0.0001 .............................................
[CV] ................. C=0.1, gamma=0.0001, score=1.000, total=   7.6s
[CV] C=0.1, gamma=0.0001 .............................................
[CV] ................. C=0.1, gamma=0.0001, score=1.000, total=   7.6s
[CV] C=0.1, gamma=0.0001 .............................................
[CV] ................. C=0.1, gamma=0.0001, score=1.000, total=   7.7s
[CV] C=0.1, gamma=0.0001 .............................................
[CV] ................. C=0.1, gamma=0.0001, score=1.000, total=   7.8s
[CV] C=0.1, gamma=0.0001 .............................................
[CV] ................. C=0.1, gamma=0.0001, score=1.000, total=   7.7s
[CV] C=0.1, gamma=0.001 ..............................................
[CV] .

[CV] ................... C=10, gamma=1e-05, score=0.998, total=   9.9s
[CV] C=10, gamma=1e-05 ...............................................
[CV] ................... C=10, gamma=1e-05, score=0.998, total=   9.7s
[CV] C=10, gamma=1e-05 ...............................................
[CV] ................... C=10, gamma=1e-05, score=0.997, total=   9.6s
[CV] C=10, gamma=1e-05 ...............................................
[CV] ................... C=10, gamma=1e-05, score=0.997, total=   9.8s
[CV] C=10, gamma=0.0001 ..............................................
[CV] .................. C=10, gamma=0.0001, score=0.999, total=  10.2s
[CV] C=10, gamma=0.0001 ..............................................
[CV] .................. C=10, gamma=0.0001, score=0.998, total=  10.6s
[CV] C=10, gamma=0.0001 ..............................................
[CV] .................. C=10, gamma=0.0001, score=0.998, total=  10.6s
[CV] C=10, gamma=0.0001 ..............................................
[CV] .

[Parallel(n_jobs=1)]: Done  80 out of  80 | elapsed: 11.8min finished


GridSearchCV(cv=None, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='linear', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': [0.1, 1, 5, 10],
                         'gamma': [1e-05, 0.0001, 0.001, 0.01]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='recall', verbose=4)

In [103]:
print(grid.best_params_)
print(grid.best_score_)

{'C': 0.1, 'gamma': 1e-05}
1.0


In [104]:
best = grid.best_estimator_

In [105]:
# Make predictions with the hypertuned model
predictions2 = best.predict(X_test)

In [106]:
prediction_labels = label_encoder.inverse_transform(predictions2)
result_df=pd.DataFrame({"Prediction": prediction_labels, "Actual": y_test})
result_df

,Prediction,Actual
5633,Non-Fatal Injury,Non-Fatal Injury
3745,Non-Fatal Injury,Fatal
3497,Non-Fatal Injury,Non-Fatal Injury
2305,Non-Fatal Injury,Non-Fatal Injury
14492,Non-Fatal Injury,Fatal
...,...,...
13839,Non-Fatal Injury,Non-Fatal Injury
913,Non-Fatal Injury,Fatal
2588,Non-Fatal Injury,Non-Fatal Injury
3308,Non-Fatal Injury,Non-Fatal Injury


In [107]:
# Calculate classification report
from sklearn.metrics import classification_report
print("SVM model after GridSearch-------------------------")
print(classification_report(y_test, prediction_labels,
                            target_names=["Fatal", "Non-Fatal Injury"]))

SVM model after GridSearch-------------------------
                  precision    recall  f1-score   support

           Fatal       0.00      0.00      0.00       543
Non-Fatal Injury       0.87      1.00      0.93      3481

        accuracy                           0.87      4024
       macro avg       0.43      0.50      0.46      4024
    weighted avg       0.75      0.87      0.80      4024



C:\Users\JJ\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


######## List for user to select  

In [108]:
HOUR_labels = list(HOUR_LE.classes_)
ROADCLASS_labels = list(Road_class_LE.classes_)
TRAFFCTL_labels = list(TRAFFCTL_LE.classes_)
VISIBILITY_labels = list(VISIBILITY_LE.classes_)
LIGHT_labels = list(LIGHT_LE.classes_)
RDSFCOND_labels = list(RDSFCOND_LE.classes_)

print("Features are under the 6 categories: ")
print("---------------------------------")
print(f"#1.HOUR:\n {HOUR_labels}") 
print(f"#2.ROAD CLASS:\n{ROADCLASS_labels}") 
print(f"#3.Traffic Control:\n{TRAFFCTL_labels}") 
print(f"#4.Visibility:\n{VISIBILITY_labels}") 
print(f"#5.Light:\n{LIGHT_labels}") 
print(f"#6.Road Condition:\n{RDSFCOND_labels}") 

# User can select from the drop down menu except "Other", "Unknown"
# if user select 11 for HOUR category, it means user drives or on the road b/t 11 ~ 12

Features are under the 6 categories: 
---------------------------------
#1.HOUR:
 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]
#2.ROAD CLASS:
['Collector', 'Expressway', 'Laneway', 'Local', 'Major Arterial', 'Major Arterial Ramp', 'Minor Arterial', 'Other', 'Pending', 'Unknown']
#3.Traffic Control:
['No Control', 'PXO - No Ped', 'Pedestrian Crossover', 'Police Control', 'School Guard', 'Stop Sign', 'Streetcar (Stop for)', 'Traffic Controller', 'Traffic Gate', 'Traffic Signal', 'Unknown', 'Yield Sign']
#4.Visibility:
['Clear', 'Drifting Snow', 'Fog, Mist, Smoke, Dust', 'Freezing Rain', 'Other', 'Rain', 'Snow', 'Strong wind', 'Unknown']
#5.Light:
['Dark', 'Dark, artificial', 'Dawn', 'Dawn, artificial', 'Daylight', 'Daylight, artificial', 'Dusk', 'Dusk, artificial', 'Other']
#6.Road Condition:
['Dry', 'Ice', 'Loose Sand or Gravel', 'Loose Snow', 'Other', 'Packed Snow', 'Slush', 'Spilled liquid', 'Unknown', 'Wet']
